In [1]:
import os
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
# Optional to save generator, checkpoint to your google drive
# Prevents data being erased if colab session disconnects
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

MessageError: ignored

In [ ]:
%cd /content/drive/MyDrive/Dissertation-project/GM-VQVAE

In [ ]:
! git clone https://github.com/rosinality/vq-vae-2-pytorch.git

In [ ]:
! pip install torch

## Loading data:

In [ ]:
"""
The following is an import of PyTorch libraries.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import numpy as np
#import nibabel as ni
import os, shutil
import time
import random
import pandas as pd
import numpy as np
import os
import cv2
import numpy as np
import os
import cv2
from scipy import ndimage
import torchvision.transforms.functional as TF
import random
import matplotlib.pyplot as plt

"""
Determine if any GPUs are available
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Visualization

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def display_image3d(x):
  z1,x1,y1 = x.nonzero()
  fig = plt.figure()
  ax = plt.axes(projection='3d')
  ax.scatter(x1, y1, z1, c=z1, cmap='viridis', alpha=0.7)
  plt.show()

## Data processing

In [ ]:
def crop_around_centroid(array, dim1):
  i, j = ndimage.center_of_mass(array)
  i, j = int(i), int(j)
  w = int(dim1/2)
  imin = max(0,i-w)
  imax = min(array.shape[0],i+w+1)
  jmin = max(0,j-w)
  jmax = min(array.shape[1],j+w+1)
  crop =  array[imin:imax,jmin:jmax]
  return crop

def standard_resize2d(image, dim):
  resize_x,  resize_y = dim[0], dim[1]
  img_sm = cv2.resize(image, (resize_x, resize_y), interpolation=cv2.INTER_CUBIC)

  return img_sm


def split_train_test(dir, ratio_test=0.15):
    if not os.path.exists(os.path.join(dir, "train")): os.mkdir(os.path.join(dir, "train"))
    if not os.path.exists(os.path.join(dir, "test")): os.mkdir(os.path.join(dir, "test"))

    images_list = [i for i in os.listdir(dir) if i.endswith(".nii")]

    random.shuffle(images_list)
    threshold = int(len(images_list)*ratio_test)
    train_list = images_list[:-threshold]
    test_list = images_list[-threshold:]

    for i in train_list:
        shutil.move(os.path.join(dir, i), os.path.join(dir, "train", i))
    for i in test_list:
        shutil.move(os.path.join(dir, i), os.path.join(dir, "test", i))

def save_data_to_csv(dir, z):
    pd.DataFrame(z).to_csv(dir, header=None, index=False)

def load_data_images(path, batch_size):
    filenames = [i for i in os.listdir(path) if i.endswith(".npy")] #and i.startswith("norm_023_S_0030")
    random.shuffle(filenames, random.random)
    n = 0
    while n < len(filenames):
        batch_image = []
        for i in range(n, n + batch_size):
            #print(filenames[i])
            if i >= len(filenames):
                ##n = i
                break

            image = np.load(os.path.join(path, filenames[i]), allow_pickle=True)#[1, ...]
            image = np.where(image >= 1e-3, image, 0.0)
            image = crop_around_centroid(image, dim1=240)
            image = np.pad(image, ((1,0), (1,0)), "constant", constant_values=0)
            dim = (256,256)
            image = torch.Tensor(standard_resize2d(image, dim))
            #image = random_rotate_transforms(image)
            image = torch.reshape(image, (1,1, 256, 256))
            image = (image - torch.min(image)) / (torch.max(image) - torch.min(image))
            image = torch.where(image >= 0.1, image, 0.0)
            batch_image.append(image)

        n += batch_size
        batch_image = torch.cat(batch_image, axis=0)
        yield batch_image

## Util Functions

In [ ]:
import math
import pickle

import torch
from torch import distributed as dist
from torch.utils import data


LOCAL_PROCESS_GROUP = None


def is_primary():
    return get_rank() == 0


def get_rank():
    if not dist.is_available():
        return 0

    if not dist.is_initialized():
        return 0

    return dist.get_rank()


def get_local_rank():
    if not dist.is_available():
        return 0

    if not dist.is_initialized():
        return 0

    if LOCAL_PROCESS_GROUP is None:
        raise ValueError("tensorfn.distributed.LOCAL_PROCESS_GROUP is None")

    return dist.get_rank(group=LOCAL_PROCESS_GROUP)


def synchronize():
    if not dist.is_available():
        return

    if not dist.is_initialized():
        return

    world_size = dist.get_world_size()

    if world_size == 1:
        return

    dist.barrier()


def get_world_size():
    if not dist.is_available():
        return 1

    if not dist.is_initialized():
        return 1

    return dist.get_world_size()


def all_reduce(tensor, op=dist.ReduceOp.SUM):
    world_size = get_world_size()

    if world_size == 1:
        return tensor

    dist.all_reduce(tensor, op=op)

    return tensor


def all_gather(data):
    world_size = get_world_size()

    if world_size == 1:
        return [data]

    buffer = pickle.dumps(data)
    storage = torch.ByteStorage.from_buffer(buffer)
    tensor = torch.ByteTensor(storage).to("cuda")

    local_size = torch.IntTensor([tensor.numel()]).to("cuda")
    size_list = [torch.IntTensor([1]).to("cuda") for _ in range(world_size)]
    dist.all_gather(size_list, local_size)
    size_list = [int(size.item()) for size in size_list]
    max_size = max(size_list)

    tensor_list = []
    for _ in size_list:
        tensor_list.append(torch.ByteTensor(size=(max_size,)).to("cuda"))

    if local_size != max_size:
        padding = torch.ByteTensor(size=(max_size - local_size,)).to("cuda")
        tensor = torch.cat((tensor, padding), 0)

    dist.all_gather(tensor_list, tensor)

    data_list = []

    for size, tensor in zip(size_list, tensor_list):
        buffer = tensor.cpu().numpy().tobytes()[:size]
        data_list.append(pickle.loads(buffer))

    return data_list


def reduce_dict(input_dict, average=True):
    world_size = get_world_size()

    if world_size < 2:
        return input_dict

    with torch.no_grad():
        keys = []
        values = []

        for k in sorted(input_dict.keys()):
            keys.append(k)
            values.append(input_dict[k])

        values = torch.stack(values, 0)
        dist.reduce(values, dst=0)

        if dist.get_rank() == 0 and average:
            values /= world_size

        reduced_dict = {k: v for k, v in zip(keys, values)}

    return reduced_dict


def data_sampler(dataset, shuffle, distributed):
    if distributed:
        return data.distributed.DistributedSampler(dataset, shuffle=shuffle)

    if shuffle:
        return data.RandomSampler(dataset)

    else:
        return data.SequentialSampler(dataset)

## GMM component

In [ ]:
"""
The following is an import of PyTorch libraries.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import numpy as np
#import nibabel as ni
import os, shutil
import time
import random
import pandas as pd
import numpy as np
import os
import cv2
import numpy as np
import os
import cv2
from scipy import ndimage
import torchvision.transforms.functional as TF
import random
import matplotlib.pyplot as plt
import torch
import torch.nn.init as init
from torch import nn
from torch.nn import functional as F
#from networks.Layers import *


"""
Determine if any GPUs are available
"""
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch
from torch import nn
from torch.nn import functional as F

# Flatten layer
class Flatten(nn.Module):
  def forward(self, x):
    return x.view(x.size(0), -1)

# Reshape layer
class Reshape(nn.Module):
  def __init__(self, outer_shape):
    super(Reshape, self).__init__()
    self.outer_shape = outer_shape
  def forward(self, x):
    return x.view(x.size(0), *self.outer_shape)

# Sample from the Gumbel-Softmax distribution and optionally discretize.
class GumbelSoftmax(nn.Module):

  def __init__(self, f_dim, c_dim):
    super(GumbelSoftmax, self).__init__()
    self.logits = nn.Linear(f_dim, c_dim)
    self.f_dim = f_dim
    self.c_dim = c_dim

  def sample_gumbel(self, shape, is_cuda=False, eps=1e-20):
    U = torch.rand(shape)
    if is_cuda:
      U = U.to(device)
    return -torch.log(-torch.log(U + eps) + eps)

  def gumbel_softmax_sample(self, logits, temperature):
    y = logits + self.sample_gumbel(logits.size(), logits.is_cuda)
    return F.softmax(y / temperature, dim=-1)

  def gumbel_softmax(self, logits, temperature, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    #categorical_dim = 10
    y = self.gumbel_softmax_sample(logits, temperature)

    if not hard:
        return y

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
    return y_hard

  def forward(self, x, temperature=1.0, hard=False):
    logits = self.logits(x).view(-1, self.c_dim)
    prob = F.softmax(logits, dim=-1)
    y = self.gumbel_softmax(logits, temperature, hard)
    return logits, prob, y

# Sample from a Gaussian distribution
class Gaussian(nn.Module):
  def __init__(self, in_dim, z_dim):
    super(Gaussian, self).__init__()
    self.mu = nn.Linear(in_dim, z_dim)
    self.var = nn.Linear(in_dim, z_dim)

  def reparameterize(self, mu, var):
    std = torch.sqrt(var + 1e-10)
    noise = torch.randn_like(std)
    z = mu + noise * std
    return z

  def forward(self, x):
    mu = self.mu(x)
    var = F.softplus(self.var(x))
    z = self.reparameterize(mu, var)
    return mu, var, z

# Inference Network
class InferenceNet(nn.Module):
  def __init__(self, x_dim, z_dim, y_dim):
    super(InferenceNet, self).__init__()

    # q(y|x)
    self.inference_qyx = torch.nn.ModuleList([
        nn.Linear(x_dim, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        GumbelSoftmax(512, y_dim)
    ])

    # q(z|y,x)
    self.inference_qzyx = torch.nn.ModuleList([
        nn.Linear(x_dim + y_dim, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        Gaussian(512, z_dim)
    ])

  # q(y|x)
  def qyx(self, x, temperature, hard):
    num_layers = len(self.inference_qyx)
    for i, layer in enumerate(self.inference_qyx):
      if i == num_layers - 1:
        #last layer is gumbel softmax
        x = layer(x, temperature, hard)
      else:
        x = layer(x)
    return x

  # q(z|x,y)
  def qzxy(self, x, y):
    concat = torch.cat((x, y), dim=1)
    for layer in self.inference_qzyx:
      concat = layer(concat)
    return concat

  def forward(self, x, temperature=1.0, hard=0):
    #x = Flatten(x)

    # q(y|x)
    logits, prob, y = self.qyx(x, temperature, hard)

    # q(z|x,y)
    mu, var, z = self.qzxy(x, y)

    output = {'mean': mu, 'var': var, 'gaussian': z,
              'logits': logits, 'prob_cat': prob, 'categorical': y}
    return output


# Generative Network
class GenerativeNet(nn.Module):
  def __init__(self, x_dim, z_dim, y_dim):
    super(GenerativeNet, self).__init__()

    # p(z|y)
    self.y_mu = nn.Linear(y_dim, z_dim)
    self.y_var = nn.Linear(y_dim, z_dim)
    # p(x|z)
    self.generative_pxz = torch.nn.ModuleList([
        nn.Linear(z_dim, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, x_dim),
        torch.nn.Sigmoid()
    ])

  # p(z|y)
  def pzy(self, y):
    y_mu = self.y_mu(y)
    y_var = F.softplus(self.y_var(y))
    return y_mu, y_var

  # p(x|z)
  def pxz(self, z):
    for layer in self.generative_pxz:
      z = layer(z)
    return z

  def forward(self, z, y):
    # p(z|y)
    y_mu, y_var = self.pzy(y)

    # p(x|z)
    x_rec = self.pxz(z)

    output = {'y_mean': y_mu, 'y_var': y_var, 'x_rec': x_rec}
    return output


# GMVAE Network
class GMVAENet(nn.Module):
  def __init__(self, x_dim, z_dim, y_dim):
    super(GMVAENet, self).__init__()
    self.inference = InferenceNet(x_dim, z_dim, y_dim)
    self.generative = GenerativeNet(x_dim, z_dim, y_dim)

    # weight initialization
    for m in self.modules():
      if type(m) == nn.Linear or type(m) == nn.Conv2d or type(m) == nn.ConvTranspose2d:
        torch.nn.init.xavier_normal_(m.weight)
        if m.bias.data is not None:
          init.constant_(m.bias, 0)

  def forward(self, x, temperature=1.0, hard=0):
    x = x.view(x.size(0), -1)
    out_inf = self.inference(x, temperature, hard)
    z, y = out_inf['gaussian'], out_inf['categorical']
    #out_gen = self.generative(z, y)

    # merge output
    #output = out_inf
    #for key, value in out_gen.items():
      #output[key] = value

    return z #output

## GM-VQVAE model

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

import distributed as dist_fn


# Copyright 2018 The Sonnet Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or  implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ============================================================================


# Borrowed from https://github.com/deepmind/sonnet and ported it to PyTorch


class Quantize(nn.Module):
    def __init__(self, dim, n_embed, decay=0.99, eps=1e-5):
        super().__init__()

        self.dim = dim
        self.n_embed = n_embed
        self.decay = decay
        self.eps = eps

        embed = torch.randn(dim, n_embed)
        self.register_buffer("embed", embed)
        self.register_buffer("cluster_size", torch.zeros(n_embed))
        self.register_buffer("embed_avg", embed.clone())

    def forward(self, input):
        flatten = input.reshape(-1, self.dim)
        dist = (
            flatten.pow(2).sum(1, keepdim=True)
            - 2 * flatten @ self.embed
            + self.embed.pow(2).sum(0, keepdim=True)
        )
        _, embed_ind = (-dist).max(1)
        embed_onehot = F.one_hot(embed_ind, self.n_embed).type(flatten.dtype)
        embed_ind = embed_ind.view(*input.shape[:-1])
        quantize = self.embed_code(embed_ind)

        if self.training:
            embed_onehot_sum = embed_onehot.sum(0)
            embed_sum = flatten.transpose(0, 1) @ embed_onehot

            all_reduce(embed_onehot_sum)
            all_reduce(embed_sum)

            self.cluster_size.data.mul_(self.decay).add_(
                embed_onehot_sum, alpha=1 - self.decay
            )
            self.embed_avg.data.mul_(self.decay).add_(embed_sum, alpha=1 - self.decay)
            n = self.cluster_size.sum()
            cluster_size = (
                (self.cluster_size + self.eps) / (n + self.n_embed * self.eps) * n
            )
            embed_normalized = self.embed_avg / cluster_size.unsqueeze(0)
            self.embed.data.copy_(embed_normalized)

        diff = (quantize.detach() - input).pow(2).mean()
        quantize = input + (quantize - input).detach()

        return quantize, diff, embed_ind

    def embed_code(self, embed_id):
        return F.embedding(embed_id, self.embed.transpose(0, 1))


class ResBlock(nn.Module):
    def __init__(self, in_channel, channel):
        super().__init__()

        self.conv = nn.Sequential(
            nn.ReLU(),
            nn.Conv2d(in_channel, channel, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel, in_channel, 1),
        )

    def forward(self, input):
        out = self.conv(input)
        out += input

        return out


class Encoder(nn.Module):
    def __init__(self, in_channel, channel, n_res_block, n_res_channel, stride):
        super().__init__()

        if stride == 4:
            blocks = [
                nn.Conv2d(in_channel, channel // 2, 4, stride=2, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 2, channel, 4, stride=2, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel, channel, 3, padding=1),
            ]

        elif stride == 2:
            blocks = [
                nn.Conv2d(in_channel, channel // 2, 4, stride=2, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(channel // 2, channel, 3, padding=1),
            ]

        for i in range(n_res_block):
            blocks.append(ResBlock(channel, n_res_channel))

        blocks.append(nn.ReLU(inplace=True))

        self.blocks = nn.Sequential(*blocks)

    def forward(self, input):
        return self.blocks(input)


class Decoder(nn.Module):
    def __init__(
        self, in_channel, out_channel, channel, n_res_block, n_res_channel, stride
    ):
        super().__init__()

        blocks = [nn.Conv2d(in_channel, channel, 3, padding=1)]

        for i in range(n_res_block):
            blocks.append(ResBlock(channel, n_res_channel))

        blocks.append(nn.ReLU(inplace=True))

        if stride == 4:
            blocks.extend(
                [
                    nn.ConvTranspose2d(channel, channel // 2, 4, stride=2, padding=1),
                    nn.ReLU(inplace=True),
                    nn.ConvTranspose2d(
                        channel // 2, out_channel, 4, stride=2, padding=1
                    ),
                ]
            )

        elif stride == 2:
            blocks.append(
                nn.ConvTranspose2d(channel, out_channel, 4, stride=2, padding=1)
            )

        self.blocks = nn.Sequential(*blocks)

    def forward(self, input):
        return self.blocks(input)


class VQVAE(nn.Module):
    def __init__(
        self,
        in_channel=1,
        channel=128,
        n_res_block=2,
        n_res_channel=32,
        embed_dim=64,
        n_embed=512,
        decay=0.99,
    ):
        super().__init__()
        n_embed=1024
        self.enc_b = Encoder(in_channel, channel, n_res_block, n_res_channel, stride=4)
        self.enc_t = Encoder(channel, channel, n_res_block, n_res_channel, stride=2)
        self.quantize_conv_t = nn.Conv2d(channel, embed_dim, 1)
        self.quantize_t = Quantize(embed_dim, n_embed)
        self.dec_t = Decoder(
            embed_dim, embed_dim, channel, n_res_block, n_res_channel, stride=2
        )
        self.quantize_conv_b = nn.Conv2d(embed_dim + channel, embed_dim, 1)
        self.quantize_b = Quantize(embed_dim, n_embed)
        self.upsample_t = nn.ConvTranspose2d(
            embed_dim, embed_dim, 4, stride=2, padding=1
        )
        self.dec = Decoder(
            embed_dim + embed_dim,
            in_channel,
            channel,
            n_res_block,
            n_res_channel,
            stride=4,
        )


    def forward(self, input):

        quant_t, quant_b, diff, _, _ = self.encode(input)
        #GMM  representation
        quant_t_scale, quant_b_scale = self.gmm_represent(quant_t, gaussian_size=quant_t.shape[-1]), self.gmm_represent(quant_b, gaussian_size=quant_b.shape[-1])
        #quant_t_scale, quant_b_scale = torch.mean(torch.flatten(quant_t_scale.permute(0, 2, 1, 3), start_dim=2), dim=2), torch.mean(torch.flatten(quant_b_scale.permute(0, 2, 1, 3), start_dim=2), dim=2)
        dec = self.decode(quant_t * torch.mean(quant_t_scale, dim=0).view(1, quant_t_scale.shape[1]), quant_b * torch.mean(quant_b_scale, dim=0).view(1, quant_b_scale.shape[1]))
        '''
        quant_t_scale, quant_b_scale = self.gmm_represent(quant_t, gaussian_size=quant_t.shape[-1]), self.gmm_represent(quant_b, gaussian_size=quant_b.shape[-1])
        #quant_t_scale, quant_b_scale = torch.mean(torch.flatten(quant_t_scale.permute(0, 2, 1, 3), start_dim=2), dim=2), torch.mean(torch.flatten(quant_b_scale.permute(0, 2, 1, 3), start_dim=2), dim=2)
        '''
        #dec = self.decode(quant_t * quant_t_scale, quant_b * quant_b_scale)

        return dec, diff

    def encode(self, input):
        enc_b = self.enc_b(input)
        enc_t = self.enc_t(enc_b)

        quant_t = self.quantize_conv_t(enc_t).permute(0, 2, 3, 1)
        quant_t, diff_t, id_t = self.quantize_t(quant_t)
        quant_t = quant_t.permute(0, 3, 1, 2)
        diff_t = diff_t.unsqueeze(0)

        dec_t = self.dec_t(quant_t)
        enc_b = torch.cat([dec_t, enc_b], 1)

        quant_b = self.quantize_conv_b(enc_b).permute(0, 2, 3, 1)
        quant_b, diff_b, id_b = self.quantize_b(quant_b)
        quant_b = quant_b.permute(0, 3, 1, 2)
        diff_b = diff_b.unsqueeze(0)

        return quant_t, quant_b, diff_t + diff_b, id_t, id_b

    def decode(self, quant_t, quant_b):
        upsample_t = self.upsample_t(quant_t)
        quant = torch.cat([upsample_t, quant_b], 1)
        dec = self.dec(quant)

        return dec

    def decode_code(self, code_t, code_b):
        quant_t = self.quantize_t.embed_code(code_t)
        quant_t = quant_t.permute(0, 3, 1, 2)
        quant_b = self.quantize_b.embed_code(code_b)
        quant_b = quant_b.permute(0, 3, 1, 2)

        dec = self.decode(quant_t, quant_b)

        return dec

    def gmm_represent(self, quant, gaussian_size = 64):
      # GMM for quant_t
      input_size = quant.shape[1] * quant.shape[2] * quant.shape[3]
      num_classes = 1
      gmvae = GMVAENet(input_size, gaussian_size, num_classes).to(device)
      quant_out = quant.view(quant.shape[0], 1, quant.shape[1], -1)
      quant_out = gmvae(quant_out)
      #quant_out = 2 * ((quant_out - torch.min(quant_out))/(torch.max(quant_out)-torch.min(quant_out))) - 1
      return torch.sigmoid(quant_out)

## Loss function

In [ ]:
class KLDivergence(nn.Module):
    "KL divergence between the estimated normal distribution and a prior distribution"
    def __init__(self):
        super(KLDivergence, self).__init__()
        """
        N :  the index N spans all dimensions of input
        N = H x W x D
        """
        self.N = 80*80
    def forward(self, z_mean, z_sigma):
        z_var = z_sigma * 2
        #return (1/self.N) * ( (z_mean**2 + z_var**2 - z_log_var**2 - 1).sum() )
        return 0.5 * ((z_mean**2 + z_var.exp() - z_var - 1).sum())

class L1Loss(nn.Module):

    "Measuring the `Euclidian distance` between prediction and ground truh using `L1 Norm`"
    def __init__(self):
        super(L1Loss, self).__init__()

    def forward(self, x, y):
        N = y.shape[0]*y.shape[1]*y.shape[2]*y.shape[3]*y.shape[4]
        return  ( (x - y).abs()).sum() / N

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.xavier_uniform(m.weight.data)

# Evaluation

In [ ]:
import glob

def evaluate_step(path_data_list, path_model_list, model_name_list, threshold, label_list):
  mean_dsc_list = []
  mean_hd_list = []
  for i in range(len(path_model_list)):
    mean_dsc, mean_hd = evaluate_testset(path_data_list[i], path_model_list[i], threshold, label_list[i])
    mean_dsc_list.append(mean_dsc)
    mean_hd_list.append(mean_hd)

  metric_tbl  = pd.DataFrame(list(zip(model_name_list, mean_dsc_list, mean_hd_list)), columns =['Model', 'Surface DSC', "Hausdroff Distance"])
  return metric_tbl

def evaluate_testset(path_data, path_model, threshold, label):
    vae_model = torch.load(path_model, map_location=torch.device('cpu'))
    vae_model.to(device)
    dice_list = []
    hausdroff_list = []
    gt_list = glob.glob(path_data + "/*.npy")
    for gt in gt_list:
      if label == "all":
        gt = np.load(gt)
      else:
        gt = np.load(gt)[1, ...]
      gt = (gt - np.min(gt)) / (np.max(gt) - np.min(gt))
      gt = np.where(gt >= 0.1, 1.0, 0.0)
      # interfere phrase
      vae_model.eval()
      mask = pred_roi(gt, vae_model)
      # Mask
      #display_image3d(gt)
      mask = (mask - np.min(mask)) / (np.max(mask) - np.min(mask))
      mask = mask[1, ...]
      mask_t = np.where(mask >= threshold, 1.0, 0.0)
      #display_image3d(mask_t)
      sd = calculate_surface_dsc(mask_t.astype("bool"), gt.astype("bool"))
      print(sd)
      #mask_t = np.where(mask >= 3, 1.0, 0.0)
      hd = calculate_hausoff_dist(mask_t.astype("bool"), gt.astype("bool"))
      print(hd)
      dice_list.append(sd)
      hausdroff_list.append(hd)

    print(np.mean(dice_list))
    print(np.mean(hausdroff_list))
    return np.mean(dice_list), np.mean(hausdroff_list)

def pred_roi(gt, vae_model):
    gt = np.where(gt >= 0.2, gt, 0.0)
    pred = np.zeros(gt.shape)
    with torch.no_grad():
      for i in range(gt.shape[0]):
        if np.sum(gt[i, ...]) > 0:
          slice = gt.copy()[i, ...]
          crop, imin, imax, jmin, jmax = crop_around_centroid(slice, dim1=200)
          x = load_data_images(crop)
          y, latent_loss = vae_model(x.to(device))
          y = y.detach().cpu().numpy()
          dim = crop.shape
          y = standard_resize2d(y[0, 0, ...], dim)
          pred[i, imin:imax, jmin:jmax] = y

    return gt, pred

def crop_around_centroid(array, dim1):
  i, j = ndimage.center_of_mass(array)
  i, j = int(i), int(j)
  w = int(dim1/2)
  imin = max(0,i-w)
  imax = min(array.shape[0],i+w+1)
  jmin = max(0,j-w)
  jmax = min(array.shape[1],j+w+1)
  crop =  array[imin:imax,jmin:jmax]

  return crop, imin, imax, jmin, jmax

def load_data_images(image):
  image = np.pad(image, ((1,0), (1,0)), "constant", constant_values=0)
  dim = (256,256)
  image = torch.Tensor(standard_resize2d(image, dim))
  image = torch.reshape(image, (1,1, 256, 256))
  image = (image - torch.min(image)) / (torch.max(image) - torch.min(image))
  return image

## Metric define

In [ ]:
! git clone https://github.com/deepmind/surface-distance.git
! pip install surface-distance/

Cloning into 'surface-distance'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 50 (delta 11), reused 8 (delta 7), pack-reused 33
Receiving objects: 100% (50/50), 36.67 KiB | 1.47 MiB/s, done.
Resolving deltas: 100% (24/24), done.
Processing ./surface-distance
  Preparing metadata (setup.py) ... done
  Created wheel for Surface-Distance-Based-Measures: filename=Surface_Distance_Based_Measures-0.1-py3-none-any.whl size=14338 sha256=8c276d8fcdc39b9a404b98837079d28220279ff3e33e4dc7d3aca96706307d5c
  Stored in directory: /root/.cache/pip/wheels/00/ad/02/00434d46812b7b5d417bc11d1082d04edbc0e60f37511963b7
Successfully built Surface-Distance-Based-Measures


In [ ]:
import numpy as np
import os
import pandas as pd
import surface_distance


def calculate_surface_dsc(mask_pred, mask_gt):
    surface_distances = surface_distance.compute_surface_distances(
        mask_gt, mask_pred, spacing_mm=(3, 2, 1))
    surface_dsc = surface_distance.compute_surface_dice_at_tolerance(
            surface_distances, 3)

    return surface_dsc


def calculate_surface_dsc_2mm(mask_pred, mask_gt):
    surface_distances = surface_distance.compute_surface_distances(
        mask_gt, mask_pred, spacing_mm=(3, 2, 1))
    surface_dsc = surface_distance.compute_surface_dice_at_tolerance(
            surface_distances, 2)
    return surface_dsc

def calculate_surface_dscmm(mask_pred, mask_gt):
    surface_distances = surface_distance.compute_surface_distances(
        mask_gt, mask_pred, spacing_mm=(3, 2, 1))
    surface_dsc = surface_distance.compute_surface_dice_at_tolerance(
            surface_distances, 1)
    return surface_dsc

def calculate_hausoff_dist(mask_pred, mask_gt):
    surface_distances = surface_distance.compute_surface_distances(
        mask_gt, mask_pred, spacing_mm=(3, 2, 1))
    surface_dsc = surface_distance.compute_robust_hausdorff(
            surface_distances, 100)
    return surface_dsc

def calculate_volume(voxel_array, voxel_size):
    voxel =  np.sum(voxel_array)*np.array(voxel_size)
    return voxel[0]*voxel[1]*voxel[2]

# Evaluate different experiments

## Joint

In [ ]:
## Path data list
path_data = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/joint/test/"
path_data_prostate = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/prostate/test/"
path_data_rectum = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/rectum/test/"
path_data_bladder = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/bladder/test/"

## Model list
gm_vqvae = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_prostate = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_rectum = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_bladder = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"

threshold = 0.4
model_name_list = ["GM-VQVAE", "GM-VQVAE-Prostate", "GM-VQVAE-Rectum", "GM-VQVAE-Bladder"]
path_model_list = [gm_vqvae, gm_vqvae_prostate, gm_vqvae_rectum, gm_vqvae_bladder]
path_data_list = [path_data, path_data_prostate , path_data_rectum, path_data_bladder]
roi_list = ["all", "prostate", "rectum", "bladder"]

metric_table = evaluate_step(path_data_list, path_model_list, model_name_list, threshold, roi_list)
metric_table

0.9615395862050626
29.0
0.9833120099947891
19.209372712298546
0.9451416561927514
33.37663853655727
0.924282989295789
21.37755832643195
0.9670261574443162
20.0
0.9560468383559162
25.709920264364882
0.8877743874028864
29.223278392404914
0.9523032427930676
24.515301344262525
0.9380477481869666
24.0
0.9531606489284258
21.37755832643195
0.9411584764047393
23.0
0.9124289701407489
33.95585369269929
0.945263383359371
15.524174696260024
0.9656471883362511
20.223748416156685
0.9452238059315058
24.320957479133433
0.8555451220383608
22.0
0.8703296309904822
35.90264614203248
0.9558103878833871
23.769728648009426
0.8559119946691325
32.202484376209235
0.8270710434860674
38.118237105091836
0.9398154858667919
20.322401432901575
0.8071317101969978
39.66106403010388
0.8730879107330314
30.282365962049784
0.9498153969898517
16.278820596099706
0.905238813663434
25.179356624028344
0.9111727370104878
33.66006535941367
0.9106614378621826
26.019223662515376
0.9392874722275375
18.0
0.9598463245416278
14.0
0.9320

,Model,Surface DSC,Hausdroff Distance
0,GM-VQVAE,0.945224,24.320957
1,GM-VQVAE-Prostate,0.873088,30.282366
2,GM-VQVAE-Rectum,0.934547,25.815849
3,GM-VQVAE-Bladder,0.887702,27.935557


In [ ]:
## Path data list
path_data = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/joint/test/"
path_data_prostate = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/prostate/test/"
path_data_rectum = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/rectum/test/"
path_data_bladder = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/bladder/test/"

## Model list
gm_vqvae = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_Adam_0.001_5e-7/model_vae_epoch_100.pt" #"/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_prostate = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_Adam_0.001_5e-7/model_vae_epoch_100.pt" #"/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_rectum ="/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_Adam_0.001_5e-7/model_vae_epoch_100.pt" #"/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_bladder = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_Adam_0.001_5e-7/model_vae_epoch_100.pt" #"/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"

threshold = 0.3
model_name_list = ["GM-VQVAE", "GM-VQVAE-Prostate", "GM-VQVAE-Rectum", "GM-VQVAE-Bladder"]
path_model_list = [gm_vqvae, gm_vqvae_prostate, gm_vqvae_rectum, gm_vqvae_bladder]
path_data_list = [path_data, path_data_prostate , path_data_rectum, path_data_bladder]
roi_list = ["all", "prostate", "rectum", "bladder"]

metric_table = evaluate_step(path_data_list, path_model_list, model_name_list, threshold, roi_list)
metric_table

scale shape =======================================
torch.Size([1, 32])
scale shape =======================================
torch.Size([1, 32])


KeyboardInterrupt: ignored

In [ ]:
## Path data list
path_data = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/joint/test/"
path_data_prostate = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/prostate/test/"
path_data_rectum = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/rectum/test/"
path_data_bladder = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/bladder/test/"

## Model list
gm_vqvae = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_prostate = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_rectum = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_bladder = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"

threshold = 0.3
model_name_list = ["GM-VQVAE", "GM-VQVAE-Prostate", "GM-VQVAE-Rectum", "GM-VQVAE-Bladder"]
path_model_list = [gm_vqvae, gm_vqvae_prostate, gm_vqvae_rectum, gm_vqvae_bladder]
path_data_list = [path_data, path_data_prostate , path_data_rectum, path_data_bladder]
roi_list = ["all", "prostate", "rectum", "bladder"]

metric_table = evaluate_step(path_data_list, path_model_list, model_name_list, threshold, roi_list)
metric_table

0.9812751673239842
24.269322199023193
0.9635220718781509
23.53720459187964
0.9504385109608613
32.55764119219941
0.9715910096119147
21.0
0.953500634188503
23.0
0.9317547887284271
25.0
0.9208571770207462
29.0
0.966905038268661
21.587033144922902
0.9597032087202656
25.768197453450252
0.956205552665437
21.0
0.9400244052156677
22.0
0.9366392159292488
34.52535300326414
0.9737383441047688
15.524174696260024
0.9810228301631693
19.390719429665317
0.9562269967699862
24.154260407904633
0.9101614700758194
25.475478405713993
0.8006310454175798
36.124783736376884
0.9345912565440658
26.476404589747453
0.9036060496599562
34.058772731852805
0.8652986615663051
36.013886210738214
0.9491244692920365
18.973665961010276
0.8269506264555523
42.0
0.8843376541444735
31.30328451934852
0.9379885866380083
19.6468827043885
0.9176594354721996
25.0
0.8870955066570734
32.61901286060018
0.8767746411485686
25.88435821108957
0.928011884343433
17.11724276862369
0.8852950400174906
17.0
0.9155757089937345
32.984845004941285

## Different Threshold experiments

In [ ]:
import glob

def evaluate_step(path_data_list, path_model_list, model_name_list, threshold_list, label_list):
  mean_dsc_list = []
  mean_hd_list = []
  name_list = []
  for i in range(len(path_model_list)):
    for threshold in threshold_list:
      print(model_name_list[i] + "_t" + str(threshold))
      mean_dsc, mean_hd = evaluate_testset(path_data_list[i], path_model_list[i], threshold, label_list[i])
      mean_dsc_list.append(mean_dsc)
      mean_hd_list.append(mean_hd)
      name_list.append(model_name_list[i] + "_t" + str(threshold))

  metric_tbl  = pd.DataFrame(list(zip(name_list, mean_dsc_list, mean_hd_list)), columns =['Model', 'Surface DSC', "Hausdroff Distance"])
  return metric_tbl

def evaluate_testset(path_data, path_model, threshold, label):
    vae_model = torch.load(path_model, map_location=torch.device('cpu'))
    vae_model.to(device)
    dice_list = []
    hausdroff_list = []
    gt_list = glob.glob(path_data + "/*.npy")[0:3]
    for gt in gt_list:
      if label == "all":
        gt = np.load(gt)
      else:
        gt = np.load(gt)[1, ...]
      gt = (gt - np.min(gt)) / (np.max(gt) - np.min(gt))
      gt = np.where(gt >= 0.1, 1.0, 0.0)
      # interfere phrase
      vae_model.eval()
      mask = pred_roi(gt, vae_model)
      # Mask
      #display_image3d(gt)
      mask = (mask - np.min(mask)) / (np.max(mask) - np.min(mask))
      mask = mask[1, ...]
      mask_t = np.where(mask >= threshold, 1.0, 0.0)
      #display_image3d(mask_t)
      sd = calculate_surface_dsc(mask_t.astype("bool"), gt.astype("bool"))
      print(sd)
      #mask_t = np.where(mask >= 3, 1.0, 0.0)
      hd = calculate_hausoff_dist(mask_t.astype("bool"), gt.astype("bool"))
      print(hd)
      dice_list.append(sd)
      hausdroff_list.append(hd)

    print(np.mean(dice_list))
    print(np.mean(hausdroff_list))
    return np.mean(dice_list), np.mean(hausdroff_list)

def pred_roi(gt, vae_model):
    gt = np.where(gt >= 0.2, gt, 0.0)
    pred = np.zeros(gt.shape)
    with torch.no_grad():
      for i in range(gt.shape[0]):
        if np.sum(gt[i, ...]) > 0:
          slice = gt.copy()[i, ...]
          crop, imin, imax, jmin, jmax = crop_around_centroid(slice, dim1=200)
          x = load_data_images(crop)
          y, latent_loss = vae_model(x.to(device))
          y = y.detach().cpu().numpy()
          dim = crop.shape
          y = standard_resize2d(y[0, 0, ...], dim)
          pred[i, imin:imax, jmin:jmax] = y

    return gt, pred

def crop_around_centroid(array, dim1):
  i, j = ndimage.center_of_mass(array)
  i, j = int(i), int(j)
  w = int(dim1/2)
  imin = max(0,i-w)
  imax = min(array.shape[0],i+w+1)
  jmin = max(0,j-w)
  jmax = min(array.shape[1],j+w+1)
  crop =  array[imin:imax,jmin:jmax]

  return crop, imin, imax, jmin, jmax

def load_data_images(image):
  image = np.pad(image, ((1,0), (1,0)), "constant", constant_values=0)
  dim = (256,256)
  image = torch.Tensor(standard_resize2d(image, dim))
  image = torch.reshape(image, (1,1, 256, 256))
  image = (image - torch.min(image)) / (torch.max(image) - torch.min(image))
  return image

In [ ]:
## Path data list
path_data = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/joint/test/"
path_data_prostate = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/prostate/test/"
path_data_rectum = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/rectum/test/"
path_data_bladder = "/content/drive/MyDrive/Dissertation-project/Data/featuremap-1k/bladder/test/"

## Model list
gm_vqvae = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_prostate = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_rectum = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"
gm_vqvae_bladder = "/content/drive/MyDrive/Dissertation-project/checkpoint/2d_vae/vqvae/vqvae_n1024/joint/model_vae_epoch_100.pt"


threshold_list = [0.1, 0.3, 0.5, 0.7, 0.9]
model_name_list = ["GM-VQVAE", "GM-VQVAE-Prostate", "GM-VQVAE-Rectum", "GM-VQVAE-Bladder"]
path_model_list = [gm_vqvae, gm_vqvae_prostate, gm_vqvae_rectum, gm_vqvae_bladder]
path_data_list = [path_data, path_data_prostate , path_data_rectum, path_data_bladder]
roi_list = ["all", "prostate", "rectum", "bladder"]

metric_table = evaluate_step(path_data_list, path_model_list, model_name_list, threshold_list, roi_list)
metric_table

GM-VQVAE_t0.1
0.154576712802288
200.97761069333072
0.07365173019155045
212.53940811059016
0.15171582695215957
209.9619013059274
0.12664808998199933
207.82630670328277
GM-VQVAE_t0.3
0.9515338880173213
24.0
0.9536301493640157
24.0
0.9400317284642675
31.575306807693888
0.9483985886152015
26.525102269231297
GM-VQVAE_t0.5
0.5021324636848536
29.068883707497267
0.5766111272725921
24.0
0.4848858894585591
36.05551275463989
0.521209826805335
29.708132154045717
GM-VQVAE_t0.7
0.14954693162931018
26.70205984563738
0.14214600595326254
28.0
0.14444998266424647
44.955533585978046
0.1453809734156064
33.219197810538475
GM-VQVAE_t0.9
0.0
inf
0.0
inf
0.0
inf
0.0
inf
GM-VQVAE-Prostate_t0.1
0.1652318902260577
199.88996973335105
0.21385763450198675
195.52493447128424
0.12131287089126246
195.2050204272421
0.16680079853976895
196.87330821062577
GM-VQVAE-Prostate_t0.3
0.9051756880899436
25.25866188063018
0.8408926432414381
33.0
0.9156552381288038
23.769728648009426
0.8872411898200618
27.342796842879867
GM-VQVAE

,Model,Surface DSC,Hausdroff Distance
0,GM-VQVAE_t0.1,0.126648,207.826307
1,GM-VQVAE_t0.3,0.948399,26.525102
2,GM-VQVAE_t0.5,0.521210,29.708132
3,GM-VQVAE_t0.7,0.145381,33.219198
4,GM-VQVAE_t0.9,0.000000,inf
5,GM-VQVAE-Prostate_t0.1,0.166801,196.873308
6,GM-VQVAE-Prostate_t0.3,0.887241,27.342797
7,GM-VQVAE-Prostate_t0.5,0.411814,30.775354
8,GM-VQVAE-Prostate_t0.7,0.095332,35.543578
9,GM-VQVAE-Prostate_t0.9,0.000000,inf
